In [ ]:
# LIBRERÍAS + PARÁMETROS

library(SoundShape)
library(tuneR)

IN_ROOT  <- "../results/asr_KNN/NN_AUDIO/ORIG/ASR"
OUT_ROOT <- "../results/asr_KNN/SYNTH"

TLIM     <- c(0, 5)
WL       <- 512
OVLP     <- 75
DBLEVEL  <- 25
FLIM_ALIGN <- c(0.1, 10)   # kHz

TIME_PERC  <- 0.001
TIME_PERC2 <- 0.0002       # para los que dan "time.perc too large"

FADE_MS <- 80              # rescatar

flush_now <- function() { flush.console(); Sys.sleep(0.01) }

clip16 <- function(x) pmax(pmin(x, 32767), -32768)

apply_fade <- function(w, fade_ms=80) {
  x <- as.numeric(w@left)
  n <- length(x)
  k <- as.integer(round(w@samp.rate * (fade_ms/1000)))
  if (k < 1) return(w)
  if (2*k > n) k <- floor(n/2)
  x[1:k] <- x[1:k] * seq(0, 1, length.out = k)
  x[(n-k+1):n] <- x[(n-k+1):n] * seq(1, 0, length.out = k)
  w@left <- as.integer(clip16(round(x)))
  w
}

cut_pad_to_5s_end <- function(w, fs=44100, sec=5) {
  n <- as.integer(fs * sec)
  x <- w@left
  if (length(x) >= n) w@left <- x[1:n] else w@left <- c(x, rep(0L, n - length(x)))
  w@samp.rate <- fs
  w
}


In [ ]:
# LISTA DE NODOS

node.dirs <- list.dirs(IN_ROOT, recursive = FALSE, full.names = TRUE)
node.dirs <- node.dirs[grepl("node_", basename(node.dirs))]

nodes <- gsub("node_", "", basename(node.dirs))
length(nodes)
head(nodes)


In [ ]:
# COPY + ALIGN (resume + prints)

failed <- c()

for (node in nodes) {

  src.dir <- file.path(IN_ROOT,  paste0("node_", node))
  out.dir <- file.path(OUT_ROOT, paste0("node_", node))
  wav.at  <- file.path(out.dir, "wav.at")
  ali.dir <- file.path(wav.at, "Aligned")

  dir.create(wav.at, recursive = TRUE, showWarnings = FALSE)

  src.wavs <- list.files(src.dir, pattern="\\.wav$", full.names=TRUE)
  n_expected <- length(src.wavs)

  n_in_wavat   <- length(list.files(wav.at,  pattern="\\.wav$", full.names=TRUE))
  n_in_aligned <- length(list.files(ali.dir, pattern="\\.wav$", full.names=TRUE))

  cat("\n--- Node", node, "---\n"); flush_now()
  cat("expected:", n_expected, "| wav.at:", n_in_wavat, "| Aligned:", n_in_aligned, "\n"); flush_now()

  if (n_in_wavat != n_expected) {
    cat("copying WAVs...\n"); flush_now()
    file.copy(src.wavs, wav.at, overwrite = TRUE)
    n_in_wavat <- length(list.files(wav.at, pattern="\\.wav$", full.names=TRUE))
    cat("wav.at now:", n_in_wavat, "\n"); flush_now()
  } else {
    cat("wav.at OK\n"); flush_now()
  }

  n_in_aligned <- length(list.files(ali.dir, pattern="\\.wav$", full.names=TRUE))
  if (n_in_aligned != n_expected) {
    cat("aligning (time.perc=", TIME_PERC, ", flim=", FLIM_ALIGN[1], "-", FLIM_ALIGN[2], "kHz)\n"); flush_now()

    unlink(ali.dir, recursive = TRUE)

    ok <- try(
      align.wave(
        wav.at      = wav.at,
        wav.to      = "Aligned",
        time.length = diff(TLIM),
        time.perc   = TIME_PERC,
        dBlevel     = DBLEVEL,
        flim        = FLIM_ALIGN,
        f           = 44100,
        wl          = WL,
        ovlp        = OVLP
      ),
      silent = TRUE
    )

    n_after <- length(list.files(ali.dir, pattern="\\.wav$", full.names=TRUE))
    cat("Aligned now:", n_after, "/", n_expected, "\n"); flush_now()

    if (inherits(ok, "try-error") || n_after != n_expected) {
      cat("FAIL\n"); flush_now()
      if (inherits(ok, "try-error")) cat(as.character(ok), "\n")
      failed <- c(failed, node)
    } else {
      cat("OK\n"); flush_now()
    }

  } else {
    cat("Aligned OK (skipping)\n"); flush_now()
  }
}

cat("\nFailed nodes:", paste(failed, collapse = ", "), "\n"); flush_now()
failed


In [ ]:
# RETRY time.perc (solo fallidos)

tp_failed <- c()

for (node in failed) {

  out.dir <- file.path(OUT_ROOT, paste0("node_", node))
  wav.at  <- file.path(out.dir, "wav.at")
  ali.dir <- file.path(wav.at, "Aligned")

  src.dir <- file.path(IN_ROOT, paste0("node_", node))
  n_expected <- length(list.files(src.dir, pattern="\\.wav$", full.names=TRUE))

  cat("\n=== RETRY node", node, "time.perc=", TIME_PERC2, "===\n"); flush_now()

  unlink(ali.dir, recursive = TRUE)

  ok <- try(
    align.wave(
      wav.at      = wav.at,
      wav.to      = "Aligned",
      time.length = diff(TLIM),
      time.perc   = TIME_PERC2,
      dBlevel     = DBLEVEL,
      flim        = FLIM_ALIGN,
      f           = 44100,
      wl          = WL,
      ovlp        = OVLP
    ),
    silent = TRUE
  )

  n_after <- length(list.files(ali.dir, pattern="\\.wav$", full.names=TRUE))
  cat("Aligned now:", n_after, "/", n_expected, "\n"); flush_now()

  if (inherits(ok, "try-error") || n_after != n_expected) {
    cat("FAIL\n"); flush_now()
    if (inherits(ok, "try-error")) cat(as.character(ok), "\n")
    tp_failed <- c(tp_failed, node)
  } else {
    cat("OK\n"); flush_now()
  }
}

cat("\nStill failing after time.perc retry:", paste(tp_failed, collapse = ", "), "\n"); flush_now()
tp_failed


In [ ]:
# FADE + ALIGN (solo nodos aún fallidos)

fade_failed <- c()

for (node in tp_failed) {

  out.dir <- file.path(OUT_ROOT, paste0("node_", node))
  wav.at  <- file.path(out.dir, "wav.at")
  ali.fin <- file.path(wav.at, "Aligned")

  tmp.at  <- file.path(out.dir, "wav.at_fade")
  tmp.ali <- file.path(tmp.at, "Aligned")

  wavs <- list.files(wav.at, pattern="\\.wav$", full.names=TRUE)
  n_expected <- length(wavs)

  cat("\n=== FADE+ALIGN node", node, "(fade", FADE_MS, "ms) ===\n"); flush_now()

  unlink(tmp.at, recursive = TRUE)
  dir.create(tmp.at, recursive = TRUE, showWarnings = FALSE)

  for (f in wavs) {
    w <- readWave(f)
    w2 <- apply_fade(w, fade_ms = FADE_MS)
    writeWave(w2, file.path(tmp.at, basename(f)))
  }

  unlink(tmp.ali, recursive = TRUE)

  ok <- try(
    align.wave(
      wav.at      = tmp.at,
      wav.to      = "Aligned",
      time.length = diff(TLIM),
      time.perc   = TIME_PERC2,
      dBlevel     = DBLEVEL,
      flim        = FLIM_ALIGN,
      f           = 44100,
      wl          = WL,
      ovlp        = OVLP
    ),
    silent = TRUE
  )

  n_tmp <- length(list.files(tmp.ali, pattern="\\.wav$", full.names=TRUE))
  cat("tmp Aligned:", n_tmp, "/", n_expected, "\n"); flush_now()

  unlink(ali.fin, recursive = TRUE)
  dir.create(ali.fin, recursive = TRUE, showWarnings = FALSE)

  if (n_tmp > 0) {
    file.copy(list.files(tmp.ali, pattern="\\.wav$", full.names=TRUE), ali.fin, overwrite = TRUE)
  }

  n_fin <- length(list.files(ali.fin, pattern="\\.wav$", full.names=TRUE))
  cat("FINAL Aligned:", n_fin, "/", n_expected, "\n"); flush_now()

  if (inherits(ok, "try-error") || n_fin != n_expected) {
    cat("FAIL\n"); flush_now()
    if (inherits(ok, "try-error")) cat(as.character(ok), "\n")
    fade_failed <- c(fade_failed, node)
  } else {
    cat("OK\n"); flush_now()
  }


}

cat("\nStill failing after FADE:", paste(fade_failed, collapse = ", "), "\n"); flush_now()
fade_failed


In [ ]:
# FORZAR ALIGNED A 5.0 s (recorte/pad SOLO al final)

for (node in nodes) {

  ali.dir <- file.path(OUT_ROOT, paste0("node_", node), "wav.at", "Aligned")
  wavs <- list.files(ali.dir, pattern="\\.wav$", full.names=TRUE)

  cat("Node", node, "- recortando:", length(wavs), "\n"); flush_now()

  for (f in wavs) {
    w <- readWave(f)
    w2 <- cut_pad_to_5s_end(w, fs=44100, sec=5)
    writeWave(w2, f)
  }
}